In [97]:
import gymnasium as gym
from stable_baselines3 import PPO 
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
import imageio
import numpy as np

In [98]:
lower_bound = 0
upper_bound = 2**31 - 1  # Max value for int32

In [99]:
env_id = "LunarLander-v2"
def make_llenv():
    env = gym.make(env_id, render_mode="rgb_array", continuous=True, enable_wind=True)
    return env

In [100]:
vec_env = make_vec_env(make_llenv, seed = np.random.randint(lower_bound, upper_bound), n_envs=6)

In [101]:
s_size = env.observation_space
print(s_size)
a_size = env.action_space
print(a_size)

Box([-90.        -90.         -5.         -5.         -3.1415927  -5.
  -0.         -0.       ], [90.        90.         5.         5.         3.1415927  5.
  1.         1.       ], (8,), float32)
Box(-1.0, 1.0, (2,), float32)


In [102]:
model = PPO(
    "MlpPolicy",
    vec_env,
    verbose=1,
    learning_rate=0.001
)

Using cpu device


In [103]:
# model.load("./ppo-continuous-1M.pt")

In [104]:
model.learn(total_timesteps=500000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 109      |
|    ep_rew_mean     | -367     |
| time/              |          |
|    fps             | 959      |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 12288    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 112          |
|    ep_rew_mean          | -339         |
| time/                   |              |
|    fps                  | 694          |
|    iterations           | 2            |
|    time_elapsed         | 35           |
|    total_timesteps      | 24576        |
| train/                  |              |
|    approx_kl            | 0.0056370683 |
|    clip_fraction        | 0.0515       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.82        |
|    explained_variance   | -0.00177     |
|    learning_r

In [108]:
# env = make_vec_env(env, seed = np.random.randint(lower_bound, upper_bound), n_envs=1)
env_fns = [lambda: gym.make(env_id, render_mode="rgb_array",  continuous=True)]
env = DummyVecEnv(env_fns)
video_path = "./ppo-lunarlander.mp4"
images = []
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    img = env.render()
    images.append(img)
    obs, rewards, done, info = env.step(action)

imageio.mimsave(video_path, [np.array(img) for i, img in enumerate(images)], fps=30)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [107]:
# model.save("./ppo-continuous-wind-500k.pt")